# Introduction to Generative Models

本章主要介绍以下内容：
- generative models, 以及和discrimitive models 之间的区别
- 使用Naïve Bayes 构建第一个generative model，当scale 增大是这个模型会fail

## 1. Introduction to generative modeling

### 1.1 什么事generative model?

Generative model 的官方定义如下：

A generative model describes how a dataset is generated, in terms of a probabilistic model. By sampling from this model, we are able to generate new data.

generative model 的流程如下：
- 首先我们需要有一个包含很多希望生成对象的training dataset，例如下图所示，我们可以有许多马的图片。每个training example可以称作一个observation。每个observation 可以有很多features，对于图像来说，每个pixel 可以看作一个feature.

- 我们的目标是: generate new sets of features that look as if they have been created using the same rules as the original data.

build a model that mimics this distribution as closely as possible and then sample from it to generate new, distinct observations that look as if they could have been included in the original training set.

<img src="figures/generative_modeling.png" alt="drawing" width="600"/>

A generative model must also be probabilistic rather than deterministic. The model must include a stochastic (random) element that influences the individual samples generated by the model.

下图展示了生成模型从2014 - 2018年的演进。

<img src="figures/gan_progress.png" alt="drawing" width="600"/>



### 1.2 Generative modeling 对比 Discriminative modeling

大部分分类问题使用的是discriminative models。例如，给一幅画，我们来预测这幅画是否是梵高的作品。

<img src="figures/discrimitive_modeling.png" alt="drawing" width="600"/>

#### discrimitive modeling
通常，discrimitive model 都需要标签，所以一般都是supervised learning. 

Discriminative modeling estimates p(y|x): the probability of a label y（是否是梵高的作品） given observation x（一幅画）.

#### generative modeling

可以用在unlabeled datasets和labeled datasets（对每个class 生成一个generative model）.

Generative modeling estimates p(x): the probability of observing observation x.
If the dataset is labeled, we can also build a generative model that estimates the distribution
p(x|y). 给一个类别（例如马），学习一个分布，然后通过sampling 生成一个sample（例如一幅马的图片）。

与discrimitive model 相比，Generative models 更难评估，因为生成的结果好坏通常是一个主管的评估，而不像discrimitive model 那样可以用一个指标来衡量。

discrimitive model 比generative model 更有商业应用场景。

### 1.3 我们为什么需要generative modeling?

1. 从纯学术和研究的角度，我们应该探讨数据是怎样生成的。more complete understanding of how the data was generated in the first place。这是一个更为困难的问题。原因是生成一个图像，可能的空间太大，而真正符合我们要求的图片又很少（例如，256*256 的图片，是一只猫）。

2. Generative model可以驱动其他领域的研究，例如强化学习。if the agent were able to simulate its environment through a generative model, it wouldn’t need to test out the strategy in the computer simulation

3. Generative model 和人体智能体更接近。

Current neuroscientific theory suggests that our perception of reality is not a highly complex
discriminative model operating on our sensory input to produce predictions of what
we are experiencing, but is instead a generative model that is trained from birth to
produce simulations of our surroundings that accurately match the future.

## 2. Probabilistic generative models

### 2.1 Generative Modeling Framework

1. 我们又一个数据集，observations X
2. 我们假设这个数据集是由一个分布生成的，记为$p_{data}$
3. Generative model $p_{model}$ 试图还原$p_{data}$（可以说$p_{model}$ 是对$p_{data}$的一个估计）
4. 我们基于$p_{model}$做sampling，如果$p_{model}$ 足够逼近$p_{data}$，那我们就可以生成十分逼真的图片

注意，这里$p_{model}$ sampling 的内容不能只是reproduce 原始数据集的数据。

#### 例子

我们有如下data：

<img src="figures/example_data.png" alt="drawing" width="600"/>

我们可能猜想，数据是基于中间有色方框区域随机采样生成的。这就是我们的$p_{model}$。 基于这个$p_{model}$， 我们可以随机选出更多的点。

<img src="figures/example_p_model.png" alt="drawing" width="600"/>

其实真正的数据是基于一个世界地图，所有人口超过100w 的城市的采样，如下图所示。因此，我们的$p_{model}$和真实的$p_{data}$相差很远（oversimplified），在这个场景下，我们生成的数据（点）有时候放在地图上就会感到很奇怪，这就是不好的生成。

- A 点，在海上。
- B 点，和已有点过于接近，也属于不好的生成
- C 点，好的生成。

<img src="figures/example_p_data.png" alt="drawing" width="600"/>

从这个例子中，我们可以看出生成模型的复杂性。如果要生成衣服图像，通常是上万维度的数据，更容易输出“不好”的结果，所以如何控制generative model 生成我们想要的内容是一个十分有挑战性的工作。

### 2.2 Probabilistic Generative Models

我们首先，不基于deep learning，而是基于probability 来构建一个generative model.

我们需要定义4 key terms: 
1. sample space
2. density function
3. parametric modeling
4. maximum likelihood estimation

#### 1. sample space
The sample space is the complete set of all values an observation x can take.

例如在上面的例子中，二维平面中所有的点就是我们的sample space。

#### 2. Probability Density Function
$p(x)$: 将采样空间中的每个点映射到一个0-1之间的值。采样空间中所有的点的density function相加为1.

在上面的例子中，在box 外是0，在box 内是固定值（取决于点的精度）。

通常，这样的density 函数有很多，通常，为了找到最合适的密度函数，我们使用parametric modeling 方法。

#### 3. Parametric Modeling
$p_{\theta}(x)$: is a family of density functions that can be described using a finite number of parameters, $\theta$.

在上面的例子中，不同的box 可以看作是不同参数控制的model.所以这个model 有四个参数，分别是左上角和右下角的坐标。

#### 4. Likelihood
$\mathscr{L}(\theta | x) = p_{\theta}(x)$ measures the plausibility of θ, given some observed point x.

如果我们有很多独立的观察样本X，我们只需要将每个样本的饿liklihood 相乘即可。
$\mathscr{L}(\theta | X) = \prod_{x \in X}p_{\theta}(x)$

乘机操作起来比价复杂，所以我们可以使用log-likelihood 代替。

$\mathscr{l}(\theta | X) = \sum_{x \in X}log p_{\theta}(x)$

#### 最大似然估计 maximum likelihood estimation
这样，我们可以找到最优的参数$\hat{\theta}$ 来maximize the likelihood of observating the dataset X.

$$\hat{\theta} = argmax_{\theta}\mathscr{L}(\theta | X)$$


### 2.3 Hello Wrodl!

我们提供50个observations of existing fashions:
<img src="figures/50_existing_fashions.png" alt="drawing" width="600"/>

目标：create new and exciting fashion trends (基于已有的fasion)
我们的目标是使用generative model 来解决问题。

图片来源：https://getavataaars.com/

对每一个图片，我们可以使用5个特征来描述它：发型，头发颜色，眼镜类型，衣服类型，衣服颜色。
通过观察50个fashion 的图片，我们发现：
1. 7种不同的发型
2. 6种不同的头发颜色
3. 3种不同的眼镜
4. 4种不同的衣服类型
5. 8种不同的衣服颜色

所以我们有7 × 6 × 3 × 4 × 8 = 4,032 种不同的特征组合。也可以看作我们有4,032 个点。通过观察，我们还可以发现，数据的分布$p_{data}$ 对某些特征的值是有偏好的，例如白色的衣服看上去就会更多一些，黑色的头发会比较少一些。可是我们并不知道$p_{data}$, 所以我们要通过这50张图片来狗仔$p_{model}$, 作为对$p_{data}$的一个估计。

我们有4,032 个点，我们要估计每个点的概率，我们需要估计4,031 个参数 ($\theta_1, ..., \theta_{4031}$)，最后一个点的概率不用估计，只需要用$1-\theta_1 - ... - \theta_{4031}$ 即可计算出。这是一个multinomial distribution，

$$\hat{\theta_j} = \frac{n_j}{N}$$

$n_j$ 第j 个combination 出现的个数， N=50.

当我们有这样一个list，其实就是一个generative model，但是问题是，如果第j个组合在训练数据集种没有出现，则$\hat{\theta_j}=0$，这样我们也无法生成相应的数据（也就是说我们只能生成已有的数据）。

#### additive smoothing
使用additive smoothing，我们把上面的公式略作修改：

$$\hat{\theta_j} = \frac{n_j + 1}{N + d}$$

然后，我们每个组合都有一个非零的概率。然而这样做仍然又一个问题：对于没有生成过的点（combination），和已有图片很像的图片与和已有图片很不一样的图片的概率值一样。而我们希望可以做到给相似的（但是未出现在训练集中的）点更高的权值。

#### Naive Bayes
NB 使用一个assumption 来很大程度的减少模型参数。

Naive Bayes assumption：每个特征都是独立的, 即：

$$p(x_j | x_i) = p(x_j)$$ 

使用这个假设，我们需要求如下概率：

$$p(x) = p(x_1, x_2, ..., x_K)$$

然后我们提贝叶斯定理：
$$p(x) = p(x_2, x_3, ..., x_K | x_1)p(x_1)$$

因为所有特征都独立，所以可以去除条件：
$$p(x) = p(x_2, x_3, ..., x_K)p(x_1)$$

然后继续使用贝叶斯定理：
$$p(x) = p(x_3, ..., x_K | x_2)p(x_2)p(x_1)$$

因为所有特征都独立，所以可以再次去除条件：
$$p(x) = p(x_3, ..., x_K )p(x_2)p(x_1)$$

一直重复以上步骤，我们有：
$$p(x) = p(x_K)...p(x_2)p(x_1)$$

写作：
$$p(x) = \prod_{k=1}^Kp(x_k)$$

所以我们的问题就简单了，我们只需要分别估计每个特征的概率即可。

$$\theta_{kl} = p(x_k=l)$$

在这个例子里，我们只需要估计5个特征每个值的出现概率，所以我们一共只需要估计7 + 6 + 3 + 4 + 8 – 5 = 23 个概率。
当我们有个combination 的时候，我们只需要把他们乘起来即可。

最大似然估计可以写作：

$$\hat{\theta_{kl}} = \frac{n_{kl}}{N}$$

$n_{kl}$ 是第k个特征的第l个取值。我们会有如下结果：

<img src="figures/nb_model_1.png" alt="drawing" width="600"/>
<img src="figures/nb_model_2.png" alt="drawing" width="600"/>

如果我们给一个组合，需要计算概率，我们只需要在上面的表格中找到相应的值然后乘起来即可。
使用这个概率表，我们可以生成新的style，例如下图所示。

<img src="figures/new_samples.png" alt="drawing" width="600"/>


## 3. The challenges of generative modeling

### 3.1 当独立同分布的假设不成立时

之前，我们的的数据集是一些特征，下面，我们只提供像素，例如，32*32 图片。所以我们有1024 个pixel，也就是1024个特征，每个特征可以取256个值。

我们给出一个新的数据集，如下图所示：

<img src="figures/planet_pixel.png" alt="drawing" width="600"/>

我们同样使用NB 模型来生成新的style，这次我们发现，我们并没有生成新的style，而是生成了10个十分相似的图片，如下图所示。

一幅图片的像素不符合naive assumption，每个像素和其旁边的像素都有关系。

图像可以生成人的形状（还有一些部位例如嘴）。这是因为在训练集种，这些avatar 的形状都差不多，而嘴的位置也大都在一个地方。而衣服的颜色我们可以看出，是很随机的，每个点都是在样本中同样位置的颜色中独立随机生成的，所以整个衣服不再有一个完整的颜色。

通过这个例子，我们可以看出generative models 有两个挑战：
1. 特征之间的dependency；
2. 每个像素一个特征，样本空间非常大，而真正符合要求的图片又很少，怎么找到这样的图片；

Deep Learning 是一种Representation Learning，可以解决上面的问题。

### 3.2 Representation Learning

当你要描述你的一个朋友长什么样，你会怎么说？高高的格子，浓浓的眉毛，身材健壮，等等。这就是representation learning.

思路是：与其直接对高纬空间建模，我们先把高纬数据投影到低维的latent space，然后学一个mapping function，给一个latent space 中的点，可以映射到原始高纬空间。latent space 中的每一个点都是高纬空间原始数据的一个representation.

例如，我们要画一个圆柱体，32*32 pixel，这样的空间比较大。我们其实只要知道两个特征即可，一个是高度，一个是宽度。所以，我们可以把所有的圆柱体投入到一个2d空间，x轴是宽度，y轴是高度，如下图所示：

<img src="figures/latent_space.png" alt="drawing" width="400"/>

在数学上的一个解释是，representation learning 的目标是在高纬空间中找到一个non-linear manifold (latent space)，然后再建立一个到原始空间的映射。如下图所示，我们假设三位空间是一个高纬空间，我们想把三维空间中的点映射到2维空间上，我们需要找到如图所示的一个latent space（扭曲，non-linear），然后找到这个变化函数。重新建立的空间的坐标轴可以理解成是可以代替这个原始数据的most relevant high-level features。

<img src="figures/latent_space_manifold.png" alt="drawing" width="400"/>

使用representation learning 的一个好处是：we can perform operations within the more manageable latent space that affect high-level properties of the image. The power of representation learning is that it actually learns which features are most important for it to describe the given observations and how to generate these features from the raw data. 所以通过latent space，我们有可能生成更“真实”的图片。


## References

[1] A Style-Based Generator Architecture for Generative Adversarial Networks

[2] Language Models Are Unsupervised Multitask Learners

[3] The Malicious Use of Artificial Intelligence: Forecasting, Prevention, and Mitigation